
## Трансформеры

## BPE


**Проблема**

 - Токенизация слов слишком высокоуровнева
 - Токенизация по буквам слишком низкоуровнева


 **Решение**

 Будем соединять токены, пока не достигнем определенного числа.

![](https://lena-voita.github.io/resources/lectures/seq2seq/bpe/build_merge_table.gif)

In [3]:
# !pip install -q youtokentome
# !pip install -q datasets

In [4]:
from datasets import load_dataset
from youtokentome import BPE

In [5]:
dataset = load_dataset("ag_news")

Using custom data configuration default
Reusing dataset ag_news (C:\Users\zald\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


In [6]:
with open("data.txt", "w") as out:
    for data_d in dataset["train"]:
        out.write(data_d["text"]+"\n")

In [7]:
data_d['text']

'Nets get Carter from Raptors INDIANAPOLIS -- All-Star Vince Carter was traded by the Toronto Raptors to the New Jersey Nets for Alonzo Mourning, Eric Williams, Aaron Williams, and a pair of first-round draft picks yesterday.'

In [8]:
bpe = BPE.train(data="data.txt", vocab_size=5000, model="bpe.bin")

In [9]:
encoded = bpe.encode("Hello there! What is BPE?")

In [10]:
tokenized = ""
for word in encoded:
    tokenized += " " + bpe.id_to_subword(word)
tokenized

' ▁H ell o ▁there ! ▁What ▁is ▁B P E ?'

## GPT 2


### Encoder vs Decoder

Так в чем разница?


![](http://jalammar.github.io/images/gpt2/self-attention-and-masked-self-attention.png)

## Авторегрессионный трансформер

![](http://jalammar.github.io/images/xlnet/gpt-2-autoregression-2.gif)

## Huggingface's transformers

![](https://raw.githubusercontent.com/huggingface/transformers/master/docs/source/imgs/transformers_logo_name.png)

In [11]:
!pip install -q transformers

In [12]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
ru_gpt = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2", output_attentions=True)

Downloading: 100%|██████████| 1.71M/1.71M [00:00<00:00, 2.47MB/s]
Downloading: 100%|██████████| 1.27M/1.27M [00:00<00:00, 2.00MB/s]
Downloading: 100%|██████████| 609/609 [00:00<00:00, 305kB/s]
Downloading: 100%|██████████| 3.14G/3.14G [04:42<00:00, 11.1MB/s]


In [14]:
device = "cuda"
ru_gpt.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [17]:
inp = """
своет моряку
"""

encoded = tokenizer.encode(inp, return_tensors="pt")


result = ru_gpt.generate(
    encoded.to(device),
    max_length=1000,
    early_stopping=True,
    no_repeat_ngram_size=5)

print(tokenizer.decode(result.cpu().flatten().numpy(), skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



своет моряку

В. И. Ленин

«…В. Ильичу, как и всем нам, не чуждо было чувство юмора. Он любил пошутить, и это было не только в его шутках, но и в его письмах, в его статьях, в его речах. Он умел шутить, и это было его главным достоинством. Он умел шутить и в то же время быть серьезным. Он умел шутить в то же время и быть серьезным. Он был человеком, который умел шутить и в тоже время быть серьезным. Это был человек, который умел шутить, но умел быть серьезным. Это было его главным достоинством».

    (Из воспоминаний В. И. Ленина)

«…Я не могу не сказать несколько слов о Владимире Ильиче Ленине. Он был человеком очень высокого роста, очень широкоплечий, с очень длинными руками. Он был очень высокого роста, но не очень широкоплечий. Он был очень широкоплечий и очень высокого роста, и он был очень высокого роста. Он был очень высок ростом, но не очень высокоплечий. Он не был высок ростом, но он был очень высокоплечий, и он был высокого роста. Он не был высокого роста, но он был высокого

# Полезные ссылки

1. https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html
2. http://jalammar.github.io/illustrated-transformer/
3. http://jalammar.github.io/illustrated-gpt2/
